# BioE 101 Lab 4 - Build an EKG

## Objectives:

- Learn about cardiac signals and EKG measurement

- Apply theory of interference, CMRR, noise, and instrumentation amplifiers

- Build filters for signal conditioning

- Gain practical experience with soldering and protoboards

- Interface conditioned analog signals with digital logic in Arduino and Python

**<span style="color:red">NOTE: This lab involves the use of electrodes directly coupled to skin. Please STOP
and CALL a TA if you are uncomfortable with using these electrodes, or if you feel
tingling/dizziness from wearing them. ALWAYS ASK if you are uncertain of a circuit
connection before hooking up power to yourself.</span>**

## Schedule and Lab Reports
-  Week 1: Prototype an EKG amplifier on a breadboard and test it on a live human
subject. Entire pre-lab is due at the beginning of the lab.

- Week 2: Read out your EKG data to the Arduino and get fancy for extra credit. The entire lab report is due the
week following the lab — no lab report is due at the beginning of the second week.

## 5. Arduino/Python Heart Monitor (Extra Credit)
- Now connect the output of your amplifier to pin A0 and the GND of your circuit to the GND pin of the Arduino board

- Build a heart monitor application which produces the following:

    - A graph of the waveform, appropriately scaled (+10 points)

    - Add a software band-pass filter to reduce interference and motion artifacts.
        (+10 points)

    -  Use an autocorrelation algorithm rather than a thresholding algorithm to
        detect heart rate. Note that this will be much more robust than the
        thresholding algorithm. (+30 points)

- To claim your extra credit, demonstrate your heart monitor to a GSI and explain the algorithms used


In [ ]:
# allows plots to be plotted right below the cell when run
%matplotlib inline 
import serial # the library for reading from serial com ports
import numpy as np # naming convention for numpy library
import matplotlib.pyplot as plt # naming convention for matplotlib
from scipy.fftpack import fft, ifft # import discrete fourier transform and its inverse
from scipy import signal

In [ ]:
def sample_arduino():
    # preamble to set up serial communications
    device = "COM9" # com port of Arduino <- CHANGE THIS TO THE ONE YOU FOUND
    baud = 115200
    ser = serial.Serial(device, baud, timeout=10)

    # read in every line of serial code and throw away corrupt data
    def read_serial_data(N):
        raw_data = []
        while (len(raw_data) < N+10): 
            data = ser.readline()
            raw_data.append(data)
        data = []
        for i in raw_data:
            try:
                data.append(float(i))
            except:
                continue
        return data
    raw_data = read_serial_data(3000) # <- Adjust this line to read in more/less data
    ser.close()
    return raw_data

In [ ]:
signal = sample_arduino()
plt.plot(signal) # change the indices to zoom in/out in time

In [ ]:
dft = fft(signal) # calculate fourier transform
fs = 1000 # sampling rate
N = len(dft) # length of discrete fourier transform
freqs = [i*fs/N for i in range(N)] # convert from dft frequencies to Hz
plt.plot(freqs[2:1000], np.abs(dft[2:1000])) # change the indices to zoom in/out in frequency

### Digital Signal Processing ###

Look up the documentation for scipy.signal's types of filters. For example, you can implement the a Butterworth Filter using:

scipy.signal.butter(N, Wn, btype='low', analog=False, output='ba')

The parameters are:

- btype : {‘lowpass’, ‘highpass’, ‘bandpass’, ‘bandstop’}

- Wn = Wcutoff/Wsample

- N : order of filter